In [1]:
import typing as T

import numpy as np
import numpy.typing as npt
import scipy as sp

from matplotlib import pyplot as plt
import matplotlib.patches as patches

from pydrake.solvers import (
    MathematicalProgram,
    MathematicalProgramResult,
    Solve,
    SolverOptions,
    CommonSolverOption, 
    IpoptSolver,
    GurobiSolver
)
from pydrake.symbolic import Polynomial, Variable, Variables, Evaluate
from sdp import create_sdp_relaxation, _get_sol_from_svd

from pydrake.math import eq, le, ge

from util import timeit, YAY, WARN, INFO, ERROR

import math

import matplotlib.pyplot as plt
import mpld3
import numpy as np
from IPython.display import HTML, display
from pydrake.all import  DiagramBuilder 
from pydrake.solvers import MathematicalProgram, Solve

# from underactuated import ConfigureParser, running_as_notebook
from underactuated import running_as_notebook
from underactuated.quadrotor2d import Quadrotor2D, Quadrotor2DVisualizer

if running_as_notebook:
    mpld3.enable_notebook()

In [2]:
builder = DiagramBuilder()
plant = builder.AddSystem(Quadrotor2D())

def get_solution_from_X(N, X, verbose = False):
    x_vec = np.real(_get_sol_from_svd(X))[1:]
    res = dict()
    N1 = N
    res["x"] = x_vec[:N1]
    res["y"] = x_vec[N1:2*N1]
    res["th"] = x_vec[2*N1:3*N1]

    res["dx"] = x_vec[3*N1:4*N1]
    res["dy"] = x_vec[4*N1:5*N1]
    res["dth"] = x_vec[5*N1:6*N1]

    res["c"] = x_vec[6*N1:7*N1]
    res["s"] = x_vec[7*N1:8*N1]

    res["dc"] = x_vec[8*N1:9*N1]
    res["ds"] = x_vec[9*N1:10*N1]

    res["v"] = x_vec[10*N1:10*N1+N]
    res["w"] = x_vec[10*N1+N:10*N1+2*N]
    if verbose:
        for name in res.keys():
            YAY( name, np.round(res[name],2) )

    INFO("ddy", (res["v"] + res["w"]) * res["c"] / plant.mass - plant.gravity)
    return res





def make_a_nonconvex_birotor_program(N, plant, init_pos=np.zeros(2), desired_pos = np.array([2,0]), dt = 0.25, custom_dt_12 = False, for_sdp_solver = False):
    # stabilize trajectory using finite horizon LQR? 
    if custom_dt_12:
        N = 12
        dt = np.array([ 0.1, 0.1, 0.1,  0.15, 0.15, 0.15,  0.2, 0.2, 0.2,  0.2, 0.2, 0.2 ])

    thrust_2_mass_ratio = 3 # 3:1 thrust : mass ratio
    r = plant.length
    m = plant.mass
    I = plant.inertia
    g = plant.gravity

    Q = np.diag([10, 10, 10,  1, 1, 20,   1, 1,1,1 ])
    Qf = Q
    R = np.array([[0.1, 0.05], [0.05, 0.1]])


    prog = MathematicalProgram()
    # define optimization variables
    # N+1 timesteps for each state
    # x = prog.NewContinuousVariables(N+1, "x")
    # y = prog.NewContinuousVariables(N+1, "y")
    # th = prog.NewContinuousVariables(N+1, "th")
    # dx = prog.NewContinuousVariables(N+1, "dx")
    # dy = prog.NewContinuousVariables(N+1, "dy")
    # dth = prog.NewContinuousVariables(N+1, "dth")
    # c = prog.NewContinuousVariables(N+1, "c")
    # s = prog.NewContinuousVariables(N+1, "s")
    x = np.hstack( ([0], prog.NewContinuousVariables(N, "x")))
    y = np.hstack( ([0], prog.NewContinuousVariables(N, "y")))
    th = np.hstack( ([0], prog.NewContinuousVariables(N, "th")))
    dx = np.hstack( ([0], prog.NewContinuousVariables(N, "dx")))
    dy = np.hstack( ([0], prog.NewContinuousVariables(N, "dy")))
    dth = np.hstack( ([0], prog.NewContinuousVariables(N, "dth")))
    c = np.hstack( ([1], prog.NewContinuousVariables(N, "c")))
    s = np.hstack( ([0], prog.NewContinuousVariables(N, "s")))
    dc = np.hstack( ([0], prog.NewContinuousVariables(N, "dc")))
    ds = np.hstack( ([0], prog.NewContinuousVariables(N, "ds")))
    # N timesteps for control inputs
    v = prog.NewContinuousVariables(N, "v")
    w = prog.NewContinuousVariables(N, "w")
    # full state and control vectors
    z = np.vstack( (x,y,th,dx,dy,dth,c,s,dc,ds)).T

    u = np.vstack( (v,w) ).T

    if for_sdp_solver:
        for n in range(1, N+1):
            # oi = 1
            prog.AddConstraint( s[n]**2 + c[n]**2 >= 0.99 )
            prog.AddConstraint( s[n]**2 + c[n]**2 <= 1.05 )
    else:
        for n in range(1, N+1):
            prog.AddConstraint( s[n]**2 + c[n]**2 >= 0.85 )
            prog.AddConstraint( s[n]**2 + c[n]**2 <= 1.1 )
        # if solving with nonlinear solver -- relax cos^2 + sin^2 = 1, add bounding boxes
        prog.AddBoundingBoxConstraint(-5*np.ones(N),  5*np.ones(N),  x[1:])
        prog.AddBoundingBoxConstraint(-5*np.ones(N),  5*np.ones(N),  y[1:])
        prog.AddBoundingBoxConstraint( -np.pi*np.ones(N),  np.pi*np.ones(N),  th[1:])
        prog.AddBoundingBoxConstraint( -10*np.ones(N),  10*np.ones(N),  dx[1:])
        prog.AddBoundingBoxConstraint( -10*np.ones(N),  10*np.ones(N),  dy[1:])
        prog.AddBoundingBoxConstraint( -np.pi*np.ones(N),  np.pi*np.ones(N),  dth[1:])

    # lower and upper bounds on control inputs
    prog.AddBoundingBoxConstraint(0, m*g/2 * thrust_2_mass_ratio, v)
    prog.AddBoundingBoxConstraint(0, m*g/2 * thrust_2_mass_ratio, w)

    # dynamics
    for n in range(N):
        prog.AddLinearEqualityConstraint( x[n+1] == x[n] + dt * dx[n]  )
        prog.AddLinearEqualityConstraint( y[n+1] == y[n] + dt * dy[n] )
        prog.AddLinearEqualityConstraint( th[n+1] == th[n] + dt * dth[n] )
        prog.AddLinearEqualityConstraint( dth[n+1] == dth[n] + dt * (v[n] - w[n]) * r / I )
        # quadratic constraints
        prog.AddConstraint( dx[n+1] == dx[n] + dt * (-(v[n] + w[n]) * s[n] / m) )
        prog.AddConstraint( dy[n+1] == dy[n] + dt * ( (v[n] + w[n]) * c[n] / m - g) )
        # prog.AddConstraint( c[n+1] == c[n] - dt * dth[n] * s[n] )
        # prog.AddConstraint( s[n+1] == s[n] + dt * dth[n] * c[n] )
        prog.AddConstraint( c[n+1] == c[n] + dt * dc[n] )
        prog.AddConstraint( s[n+1] == s[n] + dt * ds[n] )
        prog.AddConstraint( dc[n+1] == dc[n] - dt * ( r/I*(v[n]-w[n])*s[n] + dth[n]*ds[n] ) )
        prog.AddConstraint( ds[n+1] == ds[n] + dt * ( r/I*(v[n]-w[n])*c[n] + dth[n]*dc[n] ) )


    # desired point
    z_star = np.hstack( (desired_pos, np.array([0, 0,0,0, 1,0, 0,0]) ))
    u_star = m * g / 2.0 * np.array([1, 1])

    # build the cost
    cost = 0
    for i in range(N):
        cost = cost + (z[i]-z_star).dot(Q).dot(z[i]-z_star) + (u[i]-u_star).dot(R).dot(u[i]-u_star)
    cost += (z[N]-z_star).dot(Qf).dot(z[N]-z_star)
    prog.AddCost(cost)

    if for_sdp_solver:
        return prog

    print("Program built.")
    timer = timeit()
    solution = Solve(prog)
    timer.dt("Program solved.")
    
    if not solution.is_success():
        ERROR("solve failed")
    else:
        YAY("solved!")
    print(solution.get_solver_id())
    print(solution.get_optimal_cost())
    print(solution.get_solution_result())

    INFO( "x", np.round(solution.GetSolution(x[1:]),2) )
    INFO( "y",  np.round(solution.GetSolution(y[1:]),2) )
    INFO( "th", np.round(solution.GetSolution(th[1:]),2) )
    print("---")
    INFO( "dx", np.round(solution.GetSolution(dx[1:]),2) )
    INFO( "dy",  np.round(solution.GetSolution(dy[1:]),2) )
    INFO( "dth", np.round(solution.GetSolution(dth[1:]),2) )
    print("---")
    INFO( "c", np.round(solution.GetSolution(c[1:]),2) )
    INFO( "s",  np.round(solution.GetSolution(s[1:]),2) )
    print("---")
    INFO( "dc", np.round(solution.GetSolution(dc[1:]),2) )
    INFO( "ds",  np.round(solution.GetSolution(ds[1:]),2) )
    cc = np.round(solution.GetSolution(c[1:]),2) ** 2
    ss = np.round(solution.GetSolution(s[1:]),2) ** 2
    INFO( "c2+s2", cc+ss)
    print("---")
    INFO( "v", np.round(solution.GetSolution(v),2) )
    INFO( "w",  np.round(solution.GetSolution(w),2) )
    return prog, solution
    
N = 10
for_sdp_solver = True
multiply_equality_constraints = True

prog = make_a_nonconvex_birotor_program(N, plant, dt = 0.2, custom_dt_12 = False, for_sdp_solver = for_sdp_solver)

if for_sdp_solver:
    timer = timeit()
    relaxed_prog, X, basis = create_sdp_relaxation(prog, multiply_equality_constraints=multiply_equality_constraints, sample_random_equality_constraints=False, sample_percentage=0.2)
    timer.dt("SDP generation")
    relaxed_solution = Solve(relaxed_prog)
    timer.dt("SDP solving")
    print("---")
    print(relaxed_solution.is_success())
    print(relaxed_solution.get_optimal_cost())
    print(relaxed_solution.get_solution_result())
    X_val = relaxed_solution.GetSolution(X)
    eigenvals, _ = np.linalg.eig(X_val)    
    print("matrix shape", X_val.shape)
    print("Matrix rank", np.sum(eigenvals>1e-4))
    res = get_solution_from_X(N, X_val, verbose=True)
    

adding quadratic costs: 1
adding linear equality constraints: 64


KeyboardInterrupt: 

array([ 1.000e+00, -0.000e+00,  0.000e+00,  2.400e-01,  5.920e-01,
        7.810e-01,  8.820e-01,  9.370e-01,  9.660e-01,  9.810e-01,
        9.890e-01,  9.920e-01,  0.000e+00,  0.000e+00, -3.100e-02,
       -1.700e-02, -9.000e-03, -5.000e-03, -3.000e-03, -1.000e-03,
       -1.000e-03, -0.000e+00, -0.000e+00, -0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
        0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00, -0.000e+00,
        1.200e+00,  1.761e+00,  9.450e-01,  5.070e-01,  2.720e-01,
        1.450e-01,  7.700e-02,  3.900e-02,  1.600e-02,  0.000e+00,
        0.000e+00, -1.560e-01,  7.200e-02,  3.900e-02,  2.100e-02,
        1.100e-02,  6.000e-03,  3.000e-03,  2.000e-03,  1.000e-03,
       -0.000e+00, -4.000e-03, -0.000e+00, -0.000e+00, -0.000e+00,
       -0.000e+00, -0.000e+00, -0.000e+00, -0.000e+00, -0.000e+00,
       -0.000e+00,  0.000e+00,  1.019e+00,  9.980e-01,  9.750e-01,
        9.750e-01,  9.750e-01,  9.750e-01,  9.750e-01,  9.750e

array([ 1.  , -0.  , -0.  ,  0.09, -0.  , -0.  , -0.1 , -0.  ,  0.  ,
        0.  ,  0.  ,  0.43, -0.  , -0.  , -0.51, -0.  ,  0.  ,  0.  ,
        0.  ,  0.59,  0.59,  0.5 , -0.17, -0.17,  0.  ,  3.  ,  1.19,
        3.  ,  1.19])

In [16]:
I = np.eye(100)
timer = timeit()
np.outer(I[0],I[0])
# I[0].reshape(100,1) @ I[0].reshape(100,1).T
timer.dt()

0.000s since last time-check


5.888938903808594e-05

(59, 59)
35


In [8]:
x_val = X_val[:, 0]
rounded_sols = np.round(np.real(_get_sol_from_svd(X_val)),3)

In [9]:
rounded_sols

array([ 1.   , -0.   , -0.   ,  0.086, -0.   , -0.   , -0.102,  0.   ,
        0.   ,  0.   ,  0.   ,  0.428, -0.   , -0.   , -0.508, -0.   ,
        0.   ,  0.   ,  0.   ,  0.588,  0.588,  0.5  , -0.173, -0.173,
        0.   ,  3.005,  1.192,  3.005,  1.192])

In [55]:
# does the c+s constraint not hold
N2+=1
print( X_val[N2+5*N1,N2+5*N1] )
print( X_val[N2+6*N1,N2+6*N1] )

3.3079570089680914e-13
4.636539446151489


In [56]:
N = 5
N1 = 6
N2 = 7
print("x", rounded_sols[1:N2])
print("y", rounded_sols[N2: N2+N1])
print("th", rounded_sols[N2+N1: N2+2*N1])
print("dx", rounded_sols[N2+2*N1: N2+3*N1])
print("dy", rounded_sols[N2+3*N1: N2+4*N1])
print("dth", rounded_sols[N2+4*N1: N2+5*N1])
print("c", rounded_sols[N2+5*N1: N2+6*N1])
print("s", rounded_sols[N2+6*N1: N2+7*N1])
print("v", rounded_sols[N2+7*N1: N2+7*N1+N])
print("w", rounded_sols[N2+7*N1+N: N2+7*N1+2*N])

x [-0.    -0.    -0.     0.447  0.672  0.765]
y [-0.    -0.    -0.038 -0.021 -0.012 -0.009]
th [-0.  0.  0.  0.  0.  0.]
dx [-0.    -0.     2.233  1.126  0.469  0.   ]
dy [-0.    -0.19   0.085  0.043  0.018  0.   ]
dth [ 0.  0. -0.  0. -0.  0.]
c [1.  1.  0.5 0.5 0.5 0.5]
s [ 0.  0. -0. -0. -0.  0.]
v [2.153 1.192 1.192 1.192 1.192]
w [2.153 1.192 1.192 1.192 1.192]


In [27]:
# x [ 0.    0.   -0.   -0.    0.14  0.36]
# y [ 0.    0.   -0.01 -0.13 -0.01  0.06]
# th [ 0.   -0.   -0.22 -0.22 -0.22 -0.22]
# ---
# dx [-0.   -0.   -0.    0.72  1.09  1.43]
# dy [-0.   -0.05 -0.61  0.62  0.32 -0.12]
# dth [-0.   -1.12  0.    0.   -0.   -0.  ]
# ---
# v [2.28 1.74 3.89 2.01 1.85]
# w [2.37 1.65 3.89 2.01 1.85]

array([<Monomial "1">, <Monomial "x(0)">, <Monomial "x(1)">, ...,
       <Monomial "w(3)^2">, <Monomial "w(3) * w(4)">, <Monomial "w(4)^2">],
      dtype=object)

In [41]:
rounded_sols

array([ 1.   , -0.   , -0.   , -0.   ,  0.447,  0.672,  0.765, -0.   ,
       -0.   , -0.038, -0.021, -0.012, -0.009, -0.   ,  0.   ,  0.   ,
        0.   ,  0.   ,  0.   , -0.   , -0.   ,  2.233,  1.126,  0.469,
        0.   , -0.   , -0.19 ,  0.085,  0.043,  0.018,  0.   ,  0.   ,
        0.   , -0.   ,  0.   , -0.   ,  0.   ,  1.   ,  1.   ,  0.5  ,
        0.5  ,  0.5  ,  0.5  ,  0.   ,  0.   , -0.   , -0.   , -0.   ,
        0.   ,  2.153,  1.192,  1.192,  1.192,  1.192,  2.153,  1.192,
        1.192,  1.192,  1.192])

In [61]:
np.sqrt(1-0.25*0.25)

0.9682458365518543

In [3]:
class BirotorChordalProgBuilder:
    def __init__(self, N = 5, dt = 0.2, init_pos=[0,0], final_pos=[2,0]):
        # define constants
        self.thrust_2_mass_ratio = 3 # 3:1 thrust : mass ratio
        builder = DiagramBuilder()
        plant = builder.AddSystem(Quadrotor2D())
        self.r = plant.length
        self.m = plant.mass
        self.I = plant.inertia
        self.g = plant.gravity
        self.dt = dt

        # define cost matrices
        self.Q = np.array([0, 10, 10, 10,  1, 1, (self.r / 2.0 / np.pi),  10, 10 ])
        self.Qf = self.Q
        # self.R = np.array([[0.1, 0.05], [0.05, 0.1]])
        self.R1 = 0.1
        self.R12 = 0.05

        # initial / final conditions
        self.init_pos = init_pos
        self.final_pos = final_pos
        
        self.N = N
        self.n = 8
        self.m = 2
        self.d = self.n + self.m
        self.prog = MathematicalProgram()
        self.init_vars()
        self.add_dynamics()


    def init_vars(self):
        # define N d x d matrices: timesteps 0 through N-1
        self.X = []
        for i in range(self.N):
            # define a psd matrix
            mat = self.prog.NewSymmetricContinuousVariables(self.d+1, "X"+str(i))
            self.prog.AddPositiveSemidefiniteConstraint(prog)
            # X00 is 1
            self.prog.AddLinearConstraint( mat[0,0] == 1 )
            self.X.append(mat)

        # timestep N
        mat = self.prog.NewSymmetricContinuousVariables(self.n+1, "X"+str(N))
        self.prog.AddPositiveSemidefiniteConstraint(prog)
        self.prog.AddLinearConstraint( mat[0,0] == 1 )
        self.X.append( mat )

    def zero(self, expr):
        self.prog.AddLinearConstraint( eq(expr,0) )

    def pos(self, expr):
        self.prog.AddLinearConstraint( ge(expr, 0) )

    def neg(self, expr):
        self.prog.AddLinearConstraint( le(expr, 0) )

    def box(self, lb, ub, expr):
        self.prog.AddLinearConstraint( le(expr, ub) )
        self.prog.AddLinearConstraint( le(lb, expr) )

    def add_dynamics(self):
        r, m, I, g, dt = self.r, self.m, self.I, self.g, self.dt
        # add dynamics
        for n in range(self.N):
            self.zero(-self.x(n+1) + self.x(n) + dt * self.dx(n))
            self.zero(-self.y(n+1) + self.y(n) + dt * self.dy(n))
            self.zero(-self.th(n+1) + self.th(n) + dt * self.dth(n) )
            self.zero(-self.dx(n+1) + self.dx(n) - self.vs(n)*dt/m - self.ws(n)*dt/m )
            self.zero(-self.dy(n+1) + self.dy(n) + self.vc(n)*dt/m + self.wc(n)*dt/m - self.one(n)*g*dt )
            self.zero(-self.dth(n+1) + self.dth(n) + self.v(n)*dt*r/I - self.w(n)*dt*r/I )
            self.zero(-self.c(n+1) + self.c(n) - dt * self.dths(n) )
            self.zero(-self.s(n+1) + self.s(n) + dt * self.dthc(n) )

    def add_control_bounds(self):
        m, g, thrust_2_mass_ratio = self.m, self.g, self.thrust_2_mass_ratio
        for n in range(self.N):
            self.box( 0, m*g/2 * thrust_2_mass_ratio, self.v(n) )
            self.box( 0, m*g/2 * thrust_2_mass_ratio, self.w(n) )

    

    # variable ordering shenanigans
    # variable ordering in the matrices is:
    # 0 -- 1
    # 1 -- x
    # 2 -- y
    # 3 -- th
    # 4 -- dx
    # 5 -- dy
    # 6 -- dth
    # 7 -- c
    # 8 -- s
    # 9 -- v
    #10 -- w

    def one(self,i): return self.X[i][0,0]
    def x(self, i): return self.X[i][0,1]
    def y(self, i): return self.X[i][0,2]
    def th(self, i): return self.X[i][0,3]
    def dx(self, i): return self.X[i][0,4]
    def dy(self, i): return self.X[i][0,5]
    def dth(self, i): return self.X[i][0,6]
    def c(self, i): return self.X[i][0,7]
    def s(self, i): return self.X[i][0,8]
    def v(self, i): return self.X[i][0,9]
    def w(self, i): return self.X[i][0,10]

    def xx(self, i): return self.X[i][1,1]
    def yy(self, i): return self.X[i][2,2]
    def thth(self, i): return self.X[i][3,3]
    def dxdx(self, i): return self.X[i][4,4]
    def dydy(self, i): return self.X[i][5,5]
    def dthdth(self, i): return self.X[i][6,6]
    def cc(self, i): return self.X[i][7,7]
    def ss(self, i): return self.X[i][8,8]
    def vv(self, i): return self.X[i][9,9]
    def ww(self, i): return self.X[i][10,10]

    def vs(self, i):return self.X[i][9,8]
    def ws(self, i):return self.X[i][10,8]

    def vc(self, i):return self.X[i][9,7]
    def wc(self, i):return self.X[i][10,7]

    def dths(self, i):return self.X[i][6,8]
    def dthc(self, i):return self.X[i][6,7]

    def wv(self, i):return self.X[i][10,9]
    def vw(self, i):return self.X[i][10,9]
    

        







SyntaxError: invalid syntax (3361561313.py, line 29)

In [34]:
from numpy.core.umath_tests import inner1d
prog = MathematicalProgram()
n = 50
X = prog.NewSymmetricContinuousVariables(n)
x = X.flatten()
a = np.random.random((n,1))
timer = timeit()
A = []
for i in range(n):
    ei = np.zeros((n,1))
    ei[i] = 1
    A.append((ei @ a.T + a @ ei.T).flatten())
    # print(A.shape)
    # A.dot(x)
    # prog.AddLinearEqualityConstraint( e = A.dot(x), b = 0)
    # print(A)
    # print("---")
    # prog.AddLinearEqualityConstraint( e = np.sum(A * X), b = 0)

A = np.array(A)
(A.T @ A).shape
# print( (A.dot(x)).shape )
# prog.AddLinearEqualityConstraint( e = A.dot(x), b = np.zeros((n,1)))
# print(A.shape)
# timer.dt()

(2500, 2500)

In [38]:
prog = MathematicalProgram()
n = 30
m = 20
X11 = prog.NewSymmetricContinuousVariables(n, "X11")
X22 = prog.NewSymmetricContinuousVariables(m, "X22")
X33 = prog.NewSymmetricContinuousVariables(n, "X33")

X12 = prog.NewContinuousVariables(n,m,"X12")
X23 = prog.NewContinuousVariables(m,n,"X23")

X1 = np.vstack((np.hstack((X11,X12)), np.hstack((X12.T, X22))))
X3 = np.vstack((np.hstack((X22,X23)), np.hstack((X23.T, X33))))
prog.AddPositiveSemidefiniteConstraint(X1)
prog.AddPositiveSemidefiniteConstraint(X3)

A1 = np.random.random((n+m,n+m))
b1 = np.random.random()
A3 = np.random.random((n+m,n+m))
b3 = np.random.random()
prog.AddLinearConstraint( np.sum(A1 * X1) == b1 ) 
prog.AddLinearConstraint( np.sum(A3 * X3) == b3 ) 

prog.AddLinearCost( np.trace(X1) )
prog.AddLinearCost( np.trace(X3) )

timer.dt()
solution = Solve(prog)
print(solution.is_success())
print(solution.get_solution_result())

solution.

True
SolutionResult.kSolutionFound


array([0.00059368, 0.00051365, 0.00054383, ..., 0.00044684, 0.0004761 ,
       0.00045649])

In [41]:
solution.get_x_val().shape

(2340,)

In [43]:
30*31/2*2 + 20*21/2 + 30*20*2

2340.0